In [201]:
import threading
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import webbrowser

### Introduction
Model of electric engine. 
1000 - 10000 RPMs.

Differential equation describing the system.

$$J\frac{d\omega}{dt} = M_e - M_0 - M_{LOAD}$$
where:
- $M_e$: electromagnetic moment
- $M_0$: braking moment
- $M_{LOAD}$: load moment
- $I$: moment of inertia

Transformation to differential form using the Eulerian method.
$$I\frac{d\omega(t+\Delta t) - \omega (t)}{\Delta t} = M_e(t)-M_0-M_{LOAD}(t)$$
$$\omega (t+\Delta t) = \omega(t) + \frac{\Delta t}{I}(M_e(t)- M_0-M_{LOAD}(t))$$

What about $M_e(t)$?
$$U_{PI}(t) = Kp \cdot e(t) + Ki \sum^t_{k=0}e(k)\Delta t$$
$$M_e(t) = U_{PI} * const$$
$$e(t) = \omega_{ref} - \omega(t)$$

### Defined Parameters

In [202]:
# Parameters of simulation
referencedRevolutionsPerMinute = 3000

# Simulation time parameters
timeOfSimulation = 1000
timeOfSample = 0.1

# Parameters of crankshaft
brakingMoment = 0.2
loadMoment = 5
momentOfInertia = 1.2
constantOfElectromagneticMoment = 0.4 #! dostosowac

# Parameters of PI regulator
Kp = 0.007
Ki = 0.00015
Kd = 0.0015

# Constraints
Umax = 24
Umin = 0


# Lists of measured values
# TODO: Think about initalizing first elements here in list
timeOfSimulationList = [0.0]
loadMomentList = [0.0] 
electromagneticMomentList = [0.0]
adjustmentErrors = [referencedRevolutionsPerMinute]
voltagesList = [0.0]
revolutionsList = [0]

### Calculations

In [203]:
def calculateNumberOfIterations(timeOfSimulation: int, timeOfSample: float) -> int:
    """ This function calculates number of iterations for simulation of process

        @Parameters:
        - timeOfSimulation (int): total time of simulation in seconds
        - timeOfSample (float): time at which we repeat the measurement in seconds

        @Return:
        - int: number of iterations
    """
    return int(timeOfSimulation / timeOfSample) + 1

In [204]:
def calculateAdjustmentError(referencedRevolutionsPerMinute: float, currentRevolutionsPerMinute: float) -> float:
    # TODO Floats or Integers???
    """ This function calculates adjustment error which is difference between referenced value and current one

        @Parameters:
        - referencedRevolutionsPerMinute (float): set value to be obtained by regulator
        - currentRevolutionsPerMinute (float): current value

        @Return:
        - float: error
    """
    return referencedRevolutionsPerMinute - currentRevolutionsPerMinute

In [205]:
def calculateVoltageOfRegulator(errorList: list[float], iteration: int) -> float:
    """
    This function calculates current voltage of regulator using PID control.

    @Parameters:
    - errorList (list[float]): list of errors at the moment and before
    - iteration (int): information about current simulation iteration

    @Return:
    - float: current voltage of regulator
    """
    # Składnik proporcjonalny
    proportional = Kp * errorList[iteration]

    # Składnik całkowy
    integral = Ki * sum(errorList) * timeOfSample

    # Składnik różniczkowy
    if iteration > 0:  # Różniczkowanie możliwe od drugiej iteracji
        derivative = (errorList[iteration] - errorList[iteration - 1]) / timeOfSample
    else:
        derivative = 0.0  # Na pierwszej iteracji brak różniczkowego wkładu

    derivative_term = Kd * derivative

    # Suma wkładów regulatora
    voltage = proportional + integral + derivative_term


    return voltage


In [206]:
def calculateElectromagneticMoment(constant : float, currentVoltage : float) -> float:
    """ This function calculates current electromagnetic moment based on voltage of regulator

        @Parameters:
        - constant (float): used to scale moment 
        - currentVoltageOfRegulator (float): voltage of regulator at the moment

        @Return
        - float: current electromagnetic moment
    """
    return constant * currentVoltage

In [207]:
def calculateNormalizedVoltage(voltgeOfRegulator : float) -> float:
    """ This function calculates normalized voltage based on predefined constraints <0;24> [V]

    @Parameters:
    - voltageOfRegulator (float): current voltage of regulator

    @Returns:
    - float: normalized voltage used to create electromagnetic moment
    """
    return max(Umin, min(Umax, voltgeOfRegulator))

In [ ]:
def calculateRevoltions(revolutionsList : list[float], electromagneticMomentList : list[float]) -> float:
    omega = revolutionsList[-1] *(2*3.14/60)
    acceleration = (
        electromagneticMomentList[-1] - loadMoment - brakingMoment) / momentOfInertia
    newOmega=omega+timeOfSample*acceleration

    return newOmega * (60 / (2 * 3.14))

### Visualizations

In [211]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("PID Controller Simulation Visualization"),

    html.Div([
        html.Label("Load Moment"),
        dcc.Slider(
            id='slider-loadMoment', min=1, max=5, step=0.1, value=1,
            marks={i: str(i) for i in range(1, 11)}
        ),
        html.Label("Kp"),
        dcc.Slider(
            id='slider-Kp', min=0.001, max=0.02, step=0.001, value=0.007,
            marks={round(i, 3): str(round(i, 3))
                   for i in [0.001, 0.005, 0.01, 0.015, 0.02]}
        ),
        html.Label("Ki"),
        dcc.Slider(
            id='slider-Ki', min=0.00001, max=0.001, step=0.00001, value=0.00015,
            marks={round(i, 5): str(round(i, 5))
                   for i in [0.00001, 0.0001, 0.0005, 0.001]}
        ),
        html.Label("Kd"),
        dcc.Slider(
            id='slider-Kd', min=0.0001, max=0.01, step=0.0001, value=0.0015,
            marks={round(i, 4): str(round(i, 4))
                   for i in [0.0001, 0.001, 0.005, 0.01]}
        ),
    ], style={'width': '50%', 'margin': 'auto'}),

    html.Div([
        dcc.Graph(id='moments-graph'),
        dcc.Graph(id='revolutions-graph')
    ]),
])


@app.callback(
    [
        Output('moments-graph', 'figure'),
        Output('revolutions-graph', 'figure')
    ],
    [
        Input('slider-loadMoment', 'value'),
        Input('slider-Kp', 'value'),
        Input('slider-Ki', 'value'),
        Input('slider-Kd', 'value'),
    ]
)
def updateGraphs(newLoadMoment, newKp, newKi, newKd):
    # Update global parameters
    global loadMoment, Kp, Ki, Kd
    loadMoment = newLoadMoment
    Kp = newKp
    Ki = newKi
    Kd = newKd

    # Reinitialize global lists
    global timeOfSimulationList, loadMomentList, electromagneticMomentList
    global adjustmentErrors, voltagesList, revolutionsList, brakingMomentList
    timeOfSimulationList = [0.0]
    loadMomentList = [0.0]
    electromagneticMomentList = [0.0]
    adjustmentErrors = [referencedRevolutionsPerMinute]
    voltagesList = [0.0]
    revolutionsList = [0]
    brakingMomentList = [brakingMoment]

    for i in range(int(timeOfSimulation / timeOfSample)):
        timeOfSimulationList.append(timeOfSimulationList[-1] + timeOfSample)

        voltage = calculateNormalizedVoltage(
            calculateVoltageOfRegulator(adjustmentErrors, -1)
        )
        voltagesList.append(voltage)

        electromagneticMoment = calculateElectromagneticMoment(
            constantOfElectromagneticMoment, voltagesList[-1]
        )
        electromagneticMomentList.append(electromagneticMoment)

        revolutions = calculateRevoltions(
            revolutionsList, electromagneticMomentList)
        revolutionsList.append(revolutions)

        adjustmentError = calculateAdjustmentError(
            referencedRevolutionsPerMinute, revolutionsList[-1]
        )
        adjustmentErrors.append(adjustmentError)

        # Add load moment and braking moment
        loadMomentList.append(loadMoment)
        brakingMomentList.append(brakingMoment)

    # Moments graph (Load, Electromagnetic, and Braking Moment)
    moments_fig = go.Figure()

    moments_fig.add_trace(go.Scatter(
        x=timeOfSimulationList,
        y=loadMomentList,
        mode='lines',
        name='Load Moment',
        line=dict(color='blue')
    ))

    moments_fig.add_trace(go.Scatter(
        x=timeOfSimulationList,
        y=electromagneticMomentList,
        mode='lines',
        name='Electromagnetic Moment',
        line=dict(color='red')
    ))

    moments_fig.add_trace(go.Scatter(
        x=timeOfSimulationList,
        y=brakingMomentList,
        mode='lines',
        name='Braking Moment',
        line=dict(color='green')
    ))

    moments_fig.update_layout(
        title=f"Load, Electromagnetic, and Braking Moment({brakingMoment}) Over Time",
        xaxis_title="Time (s)",
        yaxis_title="Moment (Nm)"
    )

    # Revolutions graph
    revolutions_fig = go.Figure()
    revolutions_fig.add_trace(go.Scatter(
        x=timeOfSimulationList,
        y=revolutionsList,
        mode='lines',
        name='Revolutions'
    ))
    revolutions_fig.add_hline(y=3000, line_dash="dot",
                              annotation_text="Target RPM")
    revolutions_fig.update_layout(
        title="Revolutions Over Time",
        xaxis_title="Time (s)",
        yaxis_title="Revolutions per Minute (RPM)"
    )

    return moments_fig, revolutions_fig


def openBrowser():
    """Open the web browser to the Dash app"""
    webbrowser.open("http://127.0.0.1:8050")

if __name__ == '__main__':
    threading.Thread(target=lambda: app.run_server(
        debug=True, use_reloader=False, host='127.0.0.1', port=8050)).start()

    openBrowser()